# Install Required Packages

In [1]:
!pip install spacy
!pip install spacy-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 104.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 36.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyl

# Import Required Modules

In [2]:
import re
import spacy
from tqdm import tqdm # Used to display progress bar of loops
from spacy.tokens import DocBin
from spacy.util import filter_spans
from spacy_transformers import Transformer

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [ ]:
# CHANGE PATH
parent_path = "/MACCROBAT2020/" # This folder contains all of the ann and txt files

# Data Preparation
*   Extraction of lines that start with "T" from filepaths.
*   Creating Annotations and "Annotation Groups".

In [3]:
def get_annotations(ann_file_path : str , txt_file_path:str, tag_names:list):
  """
  return format : [text_file_content, (entity_label, start, end, term), ... (entity_label, start, end, term)]
  """
  # Get all file contents
  ann_fhand = open(ann_file_path, "r")
  text_fhand = open(txt_file_path, "r")

  ann_file_content = ann_fhand.read()
  text_file_content = text_fhand.read();

  ann_fhand.close()
  text_fhand.close()

  annotation_group = [text_file_content]

  # Extracting the lines that include desired labels/enities
  regex = r'^T.*(?:' + '|'.join(tag_names) + r').*$'
  all_valid = re.findall(regex , ann_file_content , flags = re.MULTILINE)

  # Processing into Annottaions
  for valid_line in all_valid:
    line_components = valid_line.split()
    entity_type,start_index, end_index , *term = line_components[1:]
    entity_type = entity_type.upper()
    term = " ".join(term)
    if(not(start_index.isdigit() and end_index.isdigit())):
      continue
    start_index = int(start_index)
    end_index = int(end_index)
    entity_type = entity_type.strip()
    annotation = (entity_type, start_index,end_index, term)

    # Add formed annotation to annotation group
    annotation_group.append(annotation)

  return annotation_group

In [4]:
fhand_filenames = open(parent_path +"all_files.txt","r") # This file contains all the names of the ann/txt file
all_filenames = fhand_filenames.readlines()

basenames = [filename[:-5] for filename in all_filenames if filename.endswith(".txt\n")]

all_annotation_groups = list()

# All desired label/entity names
tag_names = ["Sign_symptom" , "Diagnostic_procedure", "Biological_structure", "Disease_disorder","Medication"]

# Commence with the extraction of annotation
for name in tqdm(basenames):
  ann_file_path = parent_path+name+".ann"
  txt_file_path = parent_path+name+".txt"

  # Collectivise all annotations of each text file
  all_annotation_groups.append(get_annotations(ann_file_path, txt_file_path,tag_names))

100%|██████████| 200/200 [05:33<00:00,  1.67s/it]


# Data Preparation for training the model
*   Creating Spans
*   Excluding all duplicate spans

In [5]:
nlp = spacy.blank('en')
doc_bin= DocBin()

for annotation_group in tqdm(all_annotation_groups):
  content = annotation_group[0]
  entities = annotation_group[1:]

  doc = nlp.make_doc(content)
  entity_spans = list()
  for  tag_name , start_index, end_index, term in entities:
    # Make spans from indexes and labels
    span = doc.char_span(start_index , end_index, label = tag_name, alignment_mode="contract")

    if(not(span is None)):
      entity_spans.append(span)
  # Ensure that all spans are unique
  unique_spans = filter_spans(entity_spans)
  doc.set_ents(unique_spans)
  doc_bin.add(doc)
doc_bin.to_disk("train.spacy")

100%|██████████| 200/200 [00:01<00:00, 138.49it/s]


# Train the NER Model
*   Create a config file with custom settings.
*   Start the training of the model.



In [12]:
! python -m spacy init config config.cfg --lang en --pipeline ner --gpu --force

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: GPU
- Transformer: roberta-base
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [7]:
! python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy --gpu-id 0

ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
tokenizer_config.json: 100% 25.0/25.0 [00:00<00:00, 141kB/s]
config.json: 100% 481/481 [00:00<00:00, 2.91MB/s]
vocab.json: 100% 899k/899k [00:00<00:00, 1.94MB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 1.98MB/s]
tokenizer.json: 100% 1.36M/1.36M [00:00<00:00, 2.93MB/s]
model.safetensors: 100% 499M/499M [00:01<00:00, 352MB/s]
Some weights of RobertaModel were not initialized from th

# Basic testing of NER Model

In [8]:
nlp_trained_model = spacy.load("model-best")

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [9]:
doc = nlp_trained_model('''
Patient John Doe, a 45-year-old male, presented with a persistent cough, fever, and shortness of breath. The doctor ordered a chest X-ray and blood tests to diagnose the underlying condition. The X-ray revealed signs of pneumonia, and the blood work showed elevated white blood cell count, indicating an infection.

The patient was prescribed a course of Azithromycin, an antibiotic, to treat the bacterial pneumonia. Additionally, the doctor recommended taking Ibuprofen to alleviate the fever and body aches associated with the illness.

Mr. Doe has a history of hypertension and is currently on Lisinopril to manage his high blood pressure. He also has Type 2 diabetes mellitus and takes Metformin regularly to control his blood sugar levels.

During the follow-up visit, the physician noted that the patient's symptoms had improved, and the pneumonia was resolving. However, the doctor advised Mr. Doe to complete the entire course of antibiotics as prescribed to prevent a relapse.

Furthermore, the doctor recommended a pulmonary function test to evaluate the patient's lung capacity and rule out any underlying chronic respiratory conditions, such as asthma or chronic obstructive pulmonary disease (COPD).

In addition to the medical conditions, the patient reported experiencing occasional heartburn and gastric discomfort. The doctor suggested taking an over-the-counter antacid like Omeprazole to manage the symptoms of acid reflux.

Overall, with proper treatment and medication management, the patient's condition is expected to improve, and the risk of complications should be minimized.
''')

In [10]:
spacy.displacy.render(doc, style="ent", jupyter=True)

In [11]:
output_dir ="ner_model_gpu"
nlp_trained_model.to_disk(output_dir)